In [1]:
import einops
import torch

In [2]:
a = torch.arange(6).reshape(2, 3)

## Matrix Sum

In [5]:
print(a)
torch.einsum("ij ->", a)

tensor([[0, 1, 2],
        [3, 4, 5]])


tensor(15)

## Column Sum

In [6]:
torch.einsum("ij -> j", a)

tensor([3, 5, 7])

In [10]:
einops.einsum(a, "i j -> j")

tensor([3, 5, 7])

## Row Sum

In [11]:
einops.einsum(a, "i j -> i")

tensor([ 3, 12])

## Transpose

In [12]:
einops.einsum(a, "i j -> j i")

tensor([[0, 3],
        [1, 4],
        [2, 5]])

## Matrix-Vector Multiplication

In [26]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(3)
print(f"Shape of a is {a.shape}")
print(f"Shape of b is {b.shape}")

Shape of a is torch.Size([2, 3])
Shape of b is torch.Size([3])


In [27]:
print(a)

tensor([[0, 1, 2],
        [3, 4, 5]])


In [24]:
print(b)

tensor([0, 1, 2])


In [30]:
c = einops.einsum(a, b, "i j, j -> i")
print(c)

tensor([ 5, 14])


## Matrix-Matrix Multiplication

In [32]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(15).reshape(3, 5)

In [33]:
c = einops.einsum(a, b, "i j, j k -> i k")
c.shape

torch.Size([2, 5])

In [34]:
c

tensor([[ 25,  28,  31,  34,  37],
        [ 70,  82,  94, 106, 118]])

## Dot Product

### Vector

In [38]:
a = torch.arange(3)
b = torch.arange(3, 6)
print(a)
print(b)

tensor([0, 1, 2])
tensor([3, 4, 5])


In [39]:
einops.einsum(a, b, "i, i ->")

tensor(14)

### Matrix

In [44]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(6, 12).reshape(2,3)
einops.einsum(a, b, "i j, i j ->")

tensor(145)

## Hadamard Product

In [53]:
a = torch.Tensor([[3, 5, 7], [4, 9, 8]])
print(a)
b = torch.Tensor([[1, 6, 3], [0, 2, 9]])
print(b)
c = einops.einsum(a, b, "i j, i j -> i j")
print(f"Answer is: {c}")

tensor([[3., 5., 7.],
        [4., 9., 8.]])
tensor([[1., 6., 3.],
        [0., 2., 9.]])
Answer is: tensor([[ 3., 30., 21.],
        [ 0., 18., 72.]])


## Outer Product

In [54]:
a = torch.arange(3)
b = torch.arange(3,7)  # -- a vector of length 4 containing [3, 4, 5, 6]
torch.einsum('i,j->ij', [a, b])

tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])

## Batch Matrix Multiplication

In [55]:
a = torch.randn(3, 2, 5)
b = torch.randn(3, 5, 3)
einops.einsum(a, b, "b i j, b j k -> b i k")

tensor([[[-0.3785,  4.8102,  4.9211],
         [ 0.2451,  1.8521, -1.1049]],

        [[ 0.3352,  0.8699,  0.6646],
         [ 0.1678,  1.0435, -0.0347]],

        [[ 3.2729, -4.6933, -0.1536],
         [-2.4576,  2.5781,  1.6033]]])